### Name: Francesca Bottazzoli
### Project work for Foundations of Computer Science course

__Preliminary steps: import file and display a preview__

Import all needed libraries

In [1]:
import pandas as pd
import numpy as np
import re

__Import of the two datasets__

Import the first dataset and display a preview

In [2]:
playstore_data = pd.read_csv('ex-data/project/googleplaystore.csv', sep = ',', warn_bad_lines = True)

In [3]:
playstore_data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Count the number of records and columns

In [4]:
playstore_data.shape

(10841, 13)

Import the second dataset

In [5]:
user_reviews_data = pd.read_csv('ex-data/project/googleplaystore_user_reviews.csv', sep = ',', warn_bad_lines = True)

In [6]:
user_reviews_data.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


Count the number of records and columns

In [7]:
user_reviews_data.shape

(64295, 5)

__1. Convert the app sizes to a number__

Count the number of distinct values for the `Size` column

In [8]:
playstore_data.Size.nunique()

462

Display all the values to understand their format (limit to 20 is applied for displaying purposes). It seems values could terminate wht M or k, or have value: `Varies with device`

In [9]:
playstore_data.Size.value_counts().head(10)

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
15M                    184
17M                    160
19M                    154
26M                    149
16M                    149
Name: Size, dtype: int64

Create a regular expression for extracting the value number and the unit of measure and a conversion function

In [10]:
split_size_re = re.compile('(?P<value>\d+\.{0,1}\d*)(?P<UM>\w*)')

# Create a function calculating the number of byes
def Calculate_Size_In_Bytes(size, unit_measure):

    if unit_measure == 'K':
        return size*1024 
    elif unit_measure == 'M':
        return size*1048576
    elif unit_measure == 'G':
        return size*1073741824
    else:
        return size

# Size_Converter function applies split_size_re regex to split a string in a numeric part and a string part
# Then calculates the number of bytes corresponding
def Size_Converter(string):
    res = split_size_re.search(string)
    if res is None:
        return np.NaN
    else:
        value = float(res.group('value'))
        unit_measure = str.upper(res.group('UM'))
        
        return Calculate_Size_In_Bytes(value,unit_measure)  

Apply the conversion function to all the elements of `Size` column. Save the output in a new column

In [11]:
playstore_data['Size_Bytes'] = list(map(Size_Converter,playstore_data.Size))
playstore_data[['Size','Size_Bytes']].head()

,Size,Size_Bytes
0,19M,19922944.0
1,14M,14680064.0
2,8.7M,9122611.2
3,25M,26214400.0
4,2.8M,2936012.8


Check the values count for the new column

In [12]:
playstore_data.Size_Bytes.value_counts().head(10)

11534336.0    198
12582912.0    196
14680064.0    194
13631488.0    191
15728640.0    184
17825792.0    160
19922944.0    154
16777216.0    149
27262976.0    149
10485760.0    146
Name: Size_Bytes, dtype: int64

Check the converted value of `Varies with device`

In [13]:
playstore_data[playstore_data.Size == "Varies with device"]['Size_Bytes'].unique()

array([nan])

__2. Convert the number of installs to a number__

Count the unique values for the `Installs` column

In [14]:
playstore_data.Installs.nunique()

22

List all the possible values for the `Installs` column with count of occurrences

In [15]:
playstore_data.Installs.value_counts().head(10)

1,000,000+     1579
10,000,000+    1252
100,000+       1169
10,000+        1054
1,000+          907
5,000,000+      752
100+            719
500,000+        539
50,000+         479
5,000+          477
Name: Installs, dtype: int64

In [16]:
playstore_data.Installs.value_counts().tail(5)

1+                67
1,000,000,000+    58
0+                14
0                  1
Free               1
Name: Installs, dtype: int64

Most of values are made by a number and a + symbol, meaning 'more than'. 

The only exception is made by the `Free` value that seems to be an error due to a wrong formatting of the original record (a comma is missing between the first two columns).

In [17]:
playstore_data[playstore_data.Installs == 'Free']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,1.0


One possibility to fix this is to correct the wrong record in the original file, but in this case I decide to simply delete it in order not to compromise further analysis.

In [18]:
playstore_data = playstore_data.drop(10472)

In [19]:
playstore_data.Installs.nunique()

21

In [20]:
playstore_data.Installs.value_counts().tail(5)

500,000,000+      72
1+                67
1,000,000,000+    58
0+                14
0                  1
Name: Installs, dtype: int64

Create a regular expression for extracting the numeric part before the + symbol

In [21]:
installs_number_re = re.compile('(?P<value>[\d,]+)')

# Function that converts a string in format nn+ to a number nn
def Installs_Converter(string):
    res = installs_number_re.search(string)
    if res is None:
        return np.NaN
    else:
        value = int(res.group('value').replace(',',''))
        
        return value

Apply the conversion function to all the elements of `Installs` column and save the output in a new column

In [22]:
playstore_data['Installs_Number'] = list(map(Installs_Converter,playstore_data.Installs))

Compare the old-new column pairs

In [23]:
playstore_data.groupby(['Installs','Installs_Number']).size().reset_index(name='counts').sort_values(by = "counts", ascending = False)

,Installs,Installs_Number,counts
4,"1,000,000+",1000000,1579
8,"10,000,000+",10000000,1252
10,"100,000+",100000,1169
7,"10,000+",10000,1054
3,"1,000+",1000,907
14,"5,000,000+",5000000,752
9,100+,100,719
19,"500,000+",500000,539
16,"50,000+",50000,479
13,"5,000+",5000,477


Calculate the total number of unique values for the new column (originally there were 21)

In [24]:
playstore_data.Installs_Number.nunique()

20

__3. Transform “Varies with device” into a missing value__

This transformation was already applied in a previous step. Let's just check if all the occurrences have been set to `NaN`

In [25]:
playstore_data[playstore_data.Size == "Varies with device"].Size_Bytes.unique()

array([nan])

__4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)__

Count the total number of distinct values

In [26]:
playstore_data['Current Ver'].nunique()

2831

Get the list of distinct values with count of occurrences

In [27]:
playstore_data['Current Ver'].value_counts().head(10)

Varies with device    1459
1.0                    809
1.1                    264
1.2                    178
2.0                    151
1.3                    145
1.0.0                  136
1.0.1                  119
1.4                     88
1.5                     81
Name: Current Ver, dtype: int64

Create a regex that search a number in format nn.nn.nn and create a conversion function

In [28]:
version_re = re.compile('(?P<version>[\d+\.]*\d)') 

# Convert a string to format nn.nn.nn if possible
def Version_Converter(string):
    if pd.isnull(string):
        return '0'
    
    res = version_re.search(string)
    
    if res is None:
        return '0'
    else:
        return res.group('version')

Apply the conversion function to all the elements of `Current Ver` column. Save the output in a new column

In [29]:
playstore_data['Current_Ver_Updated'] = list(map(Version_Converter, playstore_data['Current Ver']))

Compare the old-new column pairs

In [30]:
playstore_data.groupby(['Current Ver','Current_Ver_Updated']).size().reset_index(name='counts').sort_values(by = "counts", ascending = False).head(10)

,Current Ver,Current_Ver_Updated,counts
2764,Varies with device,0,1459
113,1.0,1.0,809
277,1.1,1.1,264
424,1.2,1.2,178
1017,2.0,2.0,151
532,1.3,1.3,145
118,1.0.0,1.0.0,136
139,1.0.1,1.0.1,119
593,1.4,1.4,88
642,1.5,1.5,81


Total number of distinct values in the new column (originally there were 2831)

In [31]:
playstore_data.Current_Ver_Updated.nunique()

2614

Count the void values

In [32]:
sum(playstore_data.Current_Ver_Updated == np.void)

0

The number of void is greater that the one of `Varies with device` in the original column (1484 vs 1459). This is due to the fact that some values of the original column was not correctly converted

In [33]:
playstore_data.loc[(playstore_data.Current_Ver_Updated == np.void) & (playstore_data['Current Ver'] != "Varies with device"),]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Size_Bytes,Installs_Number,Current_Ver_Updated


By examining these records in detail, it seems they are correct except for the `Current Ver` column so we can keep them.

Count of the distinct values for the `Android Ver` column

In [34]:
playstore_data['Android Ver'].nunique()

33

List distinct value with cardinality

In [35]:
playstore_data['Android Ver'].value_counts().head(10)

4.1 and up            2451
4.0.3 and up          1501
4.0 and up            1375
Varies with device    1362
4.4 and up             980
2.3 and up             652
5.0 and up             601
4.2 and up             394
2.3.3 and up           281
2.2 and up             244
Name: Android Ver, dtype: int64

In [36]:
playstore_data['Android Ver'].value_counts().tail(10)

8.0 and up       6
7.1 and up       3
5.0 - 8.0        2
1.0 and up       2
4.0.3 - 7.1.1    2
2.2 - 7.1.1      1
5.0 - 7.1.1      1
5.0 - 6.0        1
7.0 - 7.1.1      1
4.1 - 7.1.1      1
Name: Android Ver, dtype: int64

Apply the same conversion function already applied to `Current Ver` column. Save the output in a new column

In [37]:
playstore_data['Android_Ver_Updated'] = list(map(Version_Converter, playstore_data['Android Ver']))
playstore_data.groupby(['Android Ver','Android_Ver_Updated']).size().reset_index(name='counts').sort_values(by = "counts", ascending = False).head(10)

,Android Ver,Android_Ver_Updated,counts
17,4.1 and up,4.1,2451
15,4.0.3 and up,4.0.3,1501
13,4.0 and up,4.0,1375
32,Varies with device,0,1362
20,4.4 and up,4.4,980
8,2.3 and up,2.3,652
25,5.0 and up,5.0,601
18,4.2 and up,4.2,394
9,2.3.3 and up,2.3.3,281
7,2.2 and up,2.2,244


Count unique values (originally they were 33)

In [38]:
playstore_data.Android_Ver_Updated.nunique()

25

Count void values (`Varies with devices` count is 1362)

In [39]:
sum(playstore_data.Android_Ver_Updated == np.void)

0

__5. Remove the duplicates__

Count of records in the original dataset

In [40]:
playstore_data.shape

(10840, 17)

Count the number of completely duplicated records

In [41]:
sum(playstore_data.duplicated())

483

Remove completely duplicated records

In [42]:
playstore_data_new = playstore_data.drop_duplicates().reset_index()

In [43]:
playstore_data_new.columns

Index(['index', 'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs',
       'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated',
       'Current Ver', 'Android Ver', 'Size_Bytes', 'Installs_Number',
       'Current_Ver_Updated', 'Android_Ver_Updated'],
      dtype='object')

Remove the "old" `Size`, `Android Ver`, `Current Ver`, `Installs number` columns from the new dataset and keep only the "normalized ones"

In [44]:
playstore_data_new = playstore_data_new.drop(columns = {'Size','Installs','Android Ver','Current Ver'})

Count of records in the new dataset

In [45]:
playstore_data_new.shape

(10357, 14)

Check if any other duplicated records exists

In [46]:
playstore_data_new.columns

Index(['index', 'App', 'Category', 'Rating', 'Reviews', 'Type', 'Price',
       'Content Rating', 'Genres', 'Last Updated', 'Size_Bytes',
       'Installs_Number', 'Current_Ver_Updated', 'Android_Ver_Updated'],
      dtype='object')

In [47]:
sum(playstore_data_new.duplicated('App'))

698

Some examples of duplicated records

In [48]:
playstore_data_new[playstore_data_new.App == 'Motorola FM Radio']

,index,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
3244,3670,Motorola FM Radio,VIDEO_PLAYERS,3.9,54807,Free,0,Everyone,Video Players & Editors,"May 2, 2018",NaN,100000000,0,0
10164,10647,Motorola FM Radio,VIDEO_PLAYERS,3.9,54815,Free,0,Everyone,Video Players & Editors,"May 2, 2018",NaN,100000000,0,0


In [49]:
playstore_data_new[playstore_data_new.App == 'ROBLOX']

,index,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
1446,1653,ROBLOX,GAME,4.5,4447388,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1494,1701,ROBLOX,GAME,4.5,4447346,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1538,1748,ROBLOX,GAME,4.5,4448791,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1625,1841,ROBLOX,GAME,4.5,4449882,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1653,1870,ROBLOX,GAME,4.5,4449910,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1786,2016,ROBLOX,FAMILY,4.5,4449910,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1857,2088,ROBLOX,FAMILY,4.5,4450855,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
1956,2206,ROBLOX,FAMILY,4.5,4450890,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1
4072,4527,ROBLOX,FAMILY,4.5,4443407,Free,0,Everyone 10+,Adventure;Action & Adventure,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1


In [50]:
playstore_data_new[playstore_data_new.App == 'Bubble Shooter']

,index,App,Category,Rating,Reviews,Type,Price,Content Rating,Genres,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
1450,1657,Bubble Shooter,GAME,4.5,148897,Free,0,Everyone,Casual,"July 17, 2018",48234496.0,10000000,1.20.1,4.0.3
1503,1711,Bubble Shooter,GAME,4.5,148895,Free,0,Everyone,Casual,"July 17, 2018",48234496.0,10000000,1.20.1,4.0.3
1658,1875,Bubble Shooter,GAME,4.5,148945,Free,0,Everyone,Casual,"July 17, 2018",48234496.0,10000000,1.20.1,4.0.3
1746,1972,Bubble Shooter,GAME,4.5,148990,Free,0,Everyone,Casual,"July 17, 2018",48234496.0,10000000,1.20.1,4.0.3
1811,2041,Bubble Shooter,FAMILY,4.4,59843,Free,0,Everyone,Puzzle;Brain Games,"January 9, 2018",20971520.0,5000000,2.3.3122,2.3
3449,3894,Bubble Shooter,GAME,4.5,43576,Free,0,Everyone,Arcade,"September 13, 2017",52428800.0,5000000,4.4,2.3


Since columns like `Category` and `Genres` could assume different values I decided to remove them from the main table and create two different datasets only for storing `App` + `Category` and `App` + `Genres` unique pairs

In [51]:
playstore_data_unique = playstore_data_new.groupby(['App']).agg({'Rating': 'max', 'Reviews': 'max', 'Type': 'max', 'Price': 'max',
       'Content Rating': 'max', 'Last Updated': 'max', 'Size_Bytes': 'max',
       'Installs_Number': 'max', 'Current_Ver_Updated': 'max', 'Android_Ver_Updated': 'max'}).reset_index()

In [52]:
playstore_data_unique[playstore_data_unique.App == 'ROBLOX']

,App,Rating,Reviews,Type,Price,Content Rating,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
7301,ROBLOX,4.5,4450890,Free,0,Everyone 10+,"July 31, 2018",70254592.0,100000000,2.347.225742,4.1


In [53]:
playstore_data_unique[playstore_data_unique.App == 'Motorola FM Radio']

,App,Rating,Reviews,Type,Price,Content Rating,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
6335,Motorola FM Radio,3.9,54815,Free,0,Everyone,"May 2, 2018",NaN,100000000,0,0


In [54]:
playstore_data_unique[playstore_data_unique.App == 'Bubble Shooter']

,App,Rating,Reviews,Type,Price,Content Rating,Last Updated,Size_Bytes,Installs_Number,Current_Ver_Updated,Android_Ver_Updated
1800,Bubble Shooter,4.5,59843,Free,0,Everyone,"September 13, 2017",52428800.0,10000000,4.4,4.0.3


In [55]:
playstore_app_category = playstore_data_new[['App','Category']].drop_duplicates()
playstore_app_genres = playstore_data_new[['App','Genres']].drop_duplicates()

In [56]:
playstore_app_category[playstore_app_category.App == 'Bubble Shooter']

,App,Category
1450,Bubble Shooter,GAME
1811,Bubble Shooter,FAMILY


In [57]:
playstore_app_genres[playstore_app_genres.App == 'Bubble Shooter']

,App,Genres
1450,Bubble Shooter,Casual
1811,Bubble Shooter,Puzzle;Brain Games
3449,Bubble Shooter,Arcade


__6. For each category, compute the number of apps__

For this task, it is possible to use the `playstore_app_category` dataframe that has already been created.

In [58]:
playstore_app_category.Category.value_counts()

FAMILY                 1909
GAME                    960
TOOLS                   829
BUSINESS                420
MEDICAL                 396
PERSONALIZATION         376
PRODUCTIVITY            374
LIFESTYLE               369
FINANCE                 345
SPORTS                  327
COMMUNICATION           316
HEALTH_AND_FITNESS      288
PHOTOGRAPHY             281
NEWS_AND_MAGAZINES      254
SOCIAL                  239
BOOKS_AND_REFERENCE     222
TRAVEL_AND_LOCAL        219
SHOPPING                202
DATING                  171
VIDEO_PLAYERS           164
MAPS_AND_NAVIGATION     131
EDUCATION               119
FOOD_AND_DRINK          112
ENTERTAINMENT           102
AUTO_AND_VEHICLES        85
LIBRARIES_AND_DEMO       84
WEATHER                  79
HOUSE_AND_HOME           74
ART_AND_DESIGN           64
EVENTS                   64
PARENTING                60
COMICS                   56
BEAUTY                   53
Name: Category, dtype: int64

__7. For each category, compute the average rating__

For this task, we can integrate the `Rating` column to the `playstore_app_category` dataframe.

In [59]:
playstore_app_category['Rating'] = playstore_app_category.merge(playstore_data_unique, left_on = 'App', right_on = 'App', how = 'inner')['Rating']

Group by `Category` and calculate the average value of `Rating` 

In [60]:
category_avg_rating = playstore_app_category.groupby(['Category'])['Rating'].mean()
category_avg_rating.head()

Category
ART_AND_DESIGN         4.264912
AUTO_AND_VEHICLES      4.300000
BEAUTY                 4.333333
BOOKS_AND_REFERENCE    4.204819
BUSINESS               4.134713
Name: Rating, dtype: float64

__8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity__

For this task, it is possible to exploit the `playstore_app_genres` dataframe that has already been generated in a previous step, simply "splitting" the values in the `Genres` columns, that contains multiple values, semicolon separated.

In [61]:
playstore_app_genres.Genres.unique()[:20]

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Art & Design;Action & Adventure',
       'Auto & Vehicles', 'Beauty', 'Books & Reference', 'Business',
       'Comics', 'Comics;Creativity', 'Communication', 'Dating',
       'Education;Education', 'Education', 'Education;Creativity',
       'Education;Music & Video', 'Education;Action & Adventure',
       'Education;Pretend Play', 'Education;Brain Games', 'Entertainment'],
      dtype=object)

Initialize two empty dataframes

In [62]:
genres = pd.DataFrame(columns={'Genre'})
app_genres = pd.DataFrame(columns={'App','Genre'})

In [63]:
for item in playstore_app_genres.itertuples():
    for i in item.Genres.split(';'):
        genres = genres.append({'Genre': i},ignore_index=True)
        app_genres = app_genres.append({'Genre': i, 'App': item.App}, ignore_index=True)

`genres` dataframe contains a lot of duplicated records that must be removed

In [64]:
genres = genres.drop_duplicates().reset_index(drop = True)
genres.shape

(53, 1)

In [65]:
app_genres = app_genres.drop_duplicates().reset_index(drop = True)
app_genres.shape

(10024, 2)

In [66]:
app_genres[app_genres.App == "Pixel Draw - Number Art Coloring Book"]

,App,Genre
5,Pixel Draw - Number Art Coloring Book,Art & Design
6,Pixel Draw - Number Art Coloring Book,Creativity


__9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)__

First I create a copy of the `app_genres` table, then I also create a copy of the `App` column that will be used for pivoting.

In [67]:
app_genres_dup = app_genres.copy()
app_genres_dup['App2'] = app_genres_dup['App']

The `pivot` function can help in counting how many times an App-Genre pair occurs in the bridge table

In [68]:
app_genres_pivot = app_genres_dup.pivot_table(values = 'App2', index = 'App', columns = ['Genre'], aggfunc = pd.Series.nunique, fill_value = 0).reset_index()
app_genres_pivot.head()

Genre,App,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,"""i DT"" Fútbol. Todos Somos Técnicos.",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,+Download 4 Instagram Twitter,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,- Free Comics - Comic Apps,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,.R,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,/u/app,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Replace 0 (meant as 0 occurrence of an `App` into a `Genre`) with `False` and values greater than 0 with `True`

In [69]:
app_genres_pivot.iloc[:,1:] = app_genres_pivot.iloc[:,1:] > 0

In [70]:
app_genres_pivot.columns

Index(['App', 'Action', 'Action & Adventure', 'Adventure', 'Arcade',
       'Art & Design', 'Auto & Vehicles', 'Beauty', 'Board',
       'Books & Reference', 'Brain Games', 'Business', 'Card', 'Casino',
       'Casual', 'Comics', 'Communication', 'Creativity', 'Dating',
       'Education', 'Educational', 'Entertainment', 'Events', 'Finance',
       'Food & Drink', 'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Maps & Navigation', 'Medical', 'Music', 'Music & Audio',
       'Music & Video', 'News & Magazines', 'Parenting', 'Personalization',
       'Photography', 'Pretend Play', 'Productivity', 'Puzzle', 'Racing',
       'Role Playing', 'Shopping', 'Simulation', 'Social', 'Sports',
       'Strategy', 'Tools', 'Travel & Local', 'Trivia',
       'Video Players & Editors', 'Weather', 'Word'],
      dtype='object', name='Genre')

In [71]:
app_genres_pivot[app_genres_pivot.App == 'Pixel Draw - Number Art Coloring Book']

Genre,App,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
7022,Pixel Draw - Number Art Coloring Book,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Merge data of the `app_genres_pivot` to the original (complete) dataframe, and generate a new dataframe

In [72]:
playstore_data_unique_genres_list = playstore_data_unique.merge(app_genres_pivot, left_on = 'App', right_on = 'App', how = 'inner')

In [73]:
playstore_data_unique_genres_list.columns

Index(['App', 'Rating', 'Reviews', 'Type', 'Price', 'Content Rating',
       'Last Updated', 'Size_Bytes', 'Installs_Number', 'Current_Ver_Updated',
       'Android_Ver_Updated', 'Action', 'Action & Adventure', 'Adventure',
       'Arcade', 'Art & Design', 'Auto & Vehicles', 'Beauty', 'Board',
       'Books & Reference', 'Brain Games', 'Business', 'Card', 'Casino',
       'Casual', 'Comics', 'Communication', 'Creativity', 'Dating',
       'Education', 'Educational', 'Entertainment', 'Events', 'Finance',
       'Food & Drink', 'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Maps & Navigation', 'Medical', 'Music', 'Music & Audio',
       'Music & Video', 'News & Magazines', 'Parenting', 'Personalization',
       'Photography', 'Pretend Play', 'Productivity', 'Puzzle', 'Racing',
       'Role Playing', 'Shopping', 'Simulation', 'Social', 'Sports',
       'Strategy', 'Tools', 'Travel & Local', 'Trivia',
       'Video Players & Editors', 'Weather', 'Word'],
     

__10. For each genre, compute the average rating. What is the genre with highest average?__

For this task, we ca integrate the `Rating` column to the `playstore_app_category` dataframe.

In [74]:
playstore_app_genres['Rating'] = playstore_app_genres.merge(playstore_data_unique, left_on = 'App', right_on = 'App', how = 'inner')['Rating']

Calculate the mean of `Rating` column grouping by `Genre`

In [75]:
genres_avg_rating = playstore_app_genres.groupby(['Genres'])['Rating'].mean()
genres_avg_rating.head()

Genres
Action                          4.198739
Action;Action & Adventure       4.150000
Adventure                       4.045283
Adventure;Action & Adventure    3.880000
Adventure;Brain Games           4.100000
Name: Rating, dtype: float64

__11. For each app, compute the approximate income, obtain as a product of number of installs and price.__

First of all we need to convert price from a string to a decimal number. In particular, we have to remove the dollar symbol

In [76]:
playstore_data_unique[playstore_data_unique.Price!='0'].Price.unique()[:20]

array(['$1.49', '$0.99', '$3.08', '$16.99', '$9.99', '$33.99', '$2.00',
       '$3.04', '$8.99', '$1.99', '$2.49', '$5.00', '$4.99', '$2.99',
       '$7.99', '$29.99', '$3.99', '$19.90', '$2.60', '$6.49'],
      dtype=object)

Create a regular expression for extracting the value number from a string in format $nn.nn and a conversion function

In [77]:
price_re = re.compile('\D*(?P<amount>\d+\.{0,1}\d*)') 

# Convert a string to format nn if possible
def Price_Converter(string):
    if pd.isnull(string):
        return np.NaN
    
    res = price_re.search(string)
    
    if res is None:
        return np.NaN
    else:
        return float(res.group('amount'))

Apply the conversion function to all the elements of `Price` column. Save the output in a new column

In [78]:
playstore_data_unique['Price_Number'] = list(map(Price_Converter,playstore_data_unique.Price))
playstore_data_unique[['Price','Price_Number']][playstore_data_unique.Price != '0'].head()

,Price,Price_Number
9,$1.49,1.49
10,$0.99,0.99
19,$3.08,3.08
23,$1.49,1.49
29,$16.99,16.99


Create an `Income` column with the multiplication of `Price` per `App` and `Installs_Number`

In [79]:
playstore_data_unique['Income'] = playstore_data_unique.Price_Number * playstore_data_unique.Installs_Number

Some checks..

In [80]:
playstore_data_unique[['Price','Installs_Number','Income']][playstore_data_unique.Income>0].head(10)

,Price,Installs_Number,Income
9,$1.49,100,149.0
10,$0.99,500,495.0
19,$3.08,1000,3080.0
23,$1.49,100,149.0
29,$16.99,1000,16990.0
34,$1.49,100,149.0
36,$0.99,10,9.9
42,$1.49,10,14.9
53,$1.49,10,14.9
90,$9.99,100,999.0


__12. For each app, compute its minimum and maximum Sentiment_polarity__

For this task, we must refer to the `user_reviews_data` dataframe

In [81]:
user_reviews_data.columns

Index(['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity'],
      dtype='object')

In [82]:
user_reviews_data.head(10)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000
5,10 Best Foods for You,Best way,Positive,1.00,0.300000
6,10 Best Foods for You,Amazing,Positive,0.60,0.900000
7,10 Best Foods for You,NaN,NaN,NaN,NaN
8,10 Best Foods for You,"Looking forward app,",Neutral,0.00,0.000000
9,10 Best Foods for You,It helpful site ! It help foods get !,Neutral,0.00,0.000000


Calculate the `min`, `max` and `count` of `Sentiment_Polarity` column for each `App`

In [83]:
user_reviews_data.groupby(['App'])['Sentiment_Polarity'].aggregate(['min', 'max', 'count']).head(10)

,min,max,count
App,,,
10 Best Foods for You,-0.800000,1.000000,194
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500,0.910000,40
11st,-1.000000,1.000000,40
1800 Contacts - Lens Store,-0.300000,0.838542,80
1LINE – One Line with One Touch,-0.825000,1.000000,38
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000,1.000000,32
21-Day Meditation Experience,-0.265625,0.587500,80
"2Date Dating App, Love and matching",-0.645833,1.000000,38
2GIS: directory & navigator,-0.375000,1.000000,40
